In [68]:
import rospy
import numpy as np
import os.path
import scipy.spatial.distance as sd
import skipthoughts
from nltk.stem.lancaster import LancasterStemmer
import nltk
stemmer = LancasterStemmer()   

ImportError: No module named rospy

In [65]:

def read_intents():
    import os
    intents_path=os.getcwd()+"/intents/";
    training_data = []
    for filename in os.listdir(intents_path):
        with open(intents_path+filename) as f:
            for line in f:
                training_data.append({"class":filename, "sentence":line})
            
    return training_data

ignore_words=['?',',','roboy','Roboy','\n','.']

def sanitize_sentence(sentence):
    words=[stemmer.stem(w.lower()) for w in sentence if w not in ignore_words]
    sentence_new= "".join(str(x) for x in words)
    return sentence_new
    
words=[]
classes=[]
documents=[]
def sanitize_dataset(training_data):
   

    #loop through each sentence in our training data
    for pattern in training_data:
        pattern['sentence']=sanitize_sentence(pattern['sentence'])
        #add to our classes list
        if pattern['class'] not in classes:
            classes.append(pattern['class'])


training_data=read_intents()
sanitize_dataset(training_data)


In [37]:
sentences=[]

for pattern in training_data:
    sentences.append(pattern['sentence'])
    


In [38]:
classes

['age_intent',
 'aim_intent',
 'city_specific_intent',
 'creator_intent',
 'event_specific_intent',
 'how_learn_intent',
 'name_intent',
 'skills_intent']

In [13]:
	model = skipthoughts.load_model()
	encoder = skipthoughts.Encoder(model)
	encodings = encoder.encode(sentences)

Loading model parameters...
Compiling encoders...


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 29 days


Loading tables...
Packing up...
3
4
5
6
7
8
9
10


In [86]:
	# A helper function to generate k nearest neighbors.
	def get_nn(sentence):
	  encoding = encoder.encode([sentence])
	  encoding = encoding[0]
	  scores = sd.cdist([encoding], encodings, "cosine")[0]
	  sorted_ids = np.argsort(scores)
	  print("Sentence : " + sentence)
	  print("\nNearest neighbors:")
	  for i in range(0, neighbors):
		print(" %d. %s (%.3f) %s" %
			  (i+1, sentences[sorted_ids[i]], scores[sorted_ids[i]],training_data[sorted_ids[i]]["class"]))
        return  training_data[sorted_ids[i]]["class"],scores[sorted_ids[i]]

In [87]:
neighbors = 1
sentence = "what year were you born"
sentence_sanitized=sanitize_sentence(sentence)
sentence_sanitized
intent=get_nn(sentence_sanitized) 
intent

5
Sentence : what year were you born

Nearest neighbors:
 1. where were you born (0.129) creator_intent


('creator_intent', 0.12915440061293348)